
## [PRÉ REQUISITO] Atividades Desenvolvidas Fora do Ambiente Databricks

|Sequência|Ação|Detalhamento
|---|---|---|
|SEQ-01|Provisionamento do Azure SQL|Provisionamento de um banco de dados no Azure|
|SEQ-02|Configuração do Ambiente Azure|Criação de toda a estrutura de tabelas e dados|


## Atividades Desenvolvidas no Notebook - 01_ingestao_dados_azure_sql

|Sequência|Ação|Detalhamento
|---|---|---|
|SEQ-01 / SEQ-02|Configuração de Biblioteca|Instalação da Biblioteca "sqlalchemy"|
|SEQ-03|Selação das Tabelas|Identificar quais serão as tabelas usadas durante o processo de ingestão|
|SEQ-04|Extração dos Dados do Azure SQL|Coleta dos Dados do Azure SQL|
|SEQ-05|Persistir os Dados em Parquet|Os dados deverão ser persistidos no diretório de cada tabela em formato parquet.|



In [0]:
pip install sqlalchemy

In [0]:
from pyspark.sql.types import* #StructType, StructField, StringType
from pyspark.sql.functions import lit, col
import pyodbc
from sqlalchemy import __version__ as sa_version, create_engine, text


In [0]:
df = spark.read \
.format("jdbc") \
.option("url", "jdbc:sqlserver://srv-db-treinamentosql.database.windows.net:1433;database=srv-db-treinamentosql") \
.option("query", "SELECT NAME AS Nome_Tabela FROM sys.tables") \
.option("user", "") \
.option("password", "") \
.load()

In [0]:
display(df)

In [0]:
lista_tabelas = df.collect()

for tabela in lista_tabelas:

    print(tabela['Nome_Tabela'])

    diretorio_parquet = "dbfs:/FileStore/tables/landing_zone/{0}".format(tabela['Nome_Tabela'])

    df = (spark.read
    .format("jdbc")
    .option("url", "jdbc:sqlserver://srv-db-treinamentosql.database.windows.net:1433;database=srv-db-treinamentosql")
    .option("query", "SELECT * FROM dbo.{0}".format(tabela['Nome_Tabela']))
    .option("user", "usr-srv-treinamentosql")
    .option("password", "42250000Gl!.")
    .load()
    )

    
    (df.write
    .format('parquet')
    .mode("overwrite")
    .save(diretorio_parquet)
    )
